## Conversion from muspy.Music to abc file - prototype
Based on ABC notation guide: http://www.lesession.co.uk/abc/abc_notation.htm 

The purpose of this prototype is to show on a simple example how the elements of the .abc file are stored in muspy.Music object and how to transform them back to .abc file.

Let's start by taking a look at an example .abc file:
> X:10 <br>
> T: Father O'Flynn <br>
> C: Trad.<br>
> O: Ireland<br>
> R: jig<br>
> M: 6/8<br>
> L: 1/8<br>
> K: Dmaj<br>
> |: A | dAF DFA | ded cBA | dcd efg | fdf ecA |<br>
> | dAF DFA | ded cBA | dcd efg fdd d2 :|

As we can see, the abc notation of a tune has two sections, the <i>header</i> with various information fields and the <i>body</i> which represents the notes of the melody.

We can read the file using mupsy read_abc function. The data read from the file is stored in muspy.Music object.

In [15]:
from typing import List


In [16]:
from muspy import read_abc

abc_music = read_abc("abc_file_0.abc")
abc_music.print()


metadata:
  schema_version: '0.2'
  title: Father O'Flynn
  creators:
  - Trad.
  source_filename: abc_file_0.abc
  source_format: abc
resolution: 24
tempos:
- time: 0
  qpm: 120.0
key_signatures:
- time: 0
  root: 2
  mode: major
  fifths: 2
- time: 516
  root: 2
  mode: major
  fifths: 2
time_signatures:
- time: 0
  numerator: 6
  denominator: 8
- time: 516
  numerator: 6
  denominator: 8
- time: 1032
  numerator: 5
  denominator: 8
barlines:
- time: 0
- time: 12
- time: 84
- time: 156
- time: 228
- time: 300
- time: 372
- time: 444
- time: 516
- time: 528
- time: 600
- time: 672
- time: 744
- time: 816
- time: 888
- time: 960
- time: 1032
beats:
- time: 0
- time: 12
- time: 24
- time: 36
- time: 48
- time: 60
- time: 72
- time: 84
- time: 96
- time: 108
- time: 120
- time: 132
- time: 144
- time: 156
- time: 168
- time: 180
- time: 192
- time: 204
- time: 216
- time: 228
- time: 240
- time: 252
- time: 264
- time: 276
- time: 288
- time: 300
- time: 312
- time: 324
- time: 336
- tim

Let's find values from the header lines in the Music object. The first line contains X: field which is put on the first line of the notation of a tune. It's and ID of a track if there is more than one song in the file. This X isn't stored in Music object, so we can give our track a new number starting from 1.

In [17]:
# list that will be containing following lines of the .abc file to write
file_lines = []

# we have only one track in this file
file_lines.append("X:1")

# the title is stored in
print(abc_music.metadata.title)
# so we can add another line
file_lines.append(f"T:{abc_music.metadata.title}")
# and another one for the composer(s) (C:)
file_lines.append(f"C:{','.join(abc_music.metadata.creators)}")


Father O'Flynn


Unfortunately, the lines about track's origin (O:), written in words rhythm (R:) and many more fields from .abc containing metadata aren't stored in the Music object, so we can't write them to our file.

The next three fields in the example file are (M: - meter), (L: - unit note length) and (K: - key). We can find their values in the Music object. 

In [18]:
# finding 6/8 meter
numerator = abc_music.time_signatures[0].numerator
denominator = abc_music.time_signatures[0].denominator
file_lines.append(f"M:{numerator}/{denominator}")

# the note length can be calculated based on the meter (here we have 1/8 - an eighth note):
file_lines.append(f"L:1/{denominator}")

# we can get the name of the note for the pitch from numeric key signature,
# music21 Pitch converts it to a string
from music21.pitch import Pitch
note = Pitch(abc_music.key_signatures[0].root)
mode = abc_music.key_signatures[0].mode
print(f'Note: {note.name+mode}')

file_lines.append(f"K:{note.name+mode}")


Note: Dmajor


Let's see how the header part of our .abc file looks like:

In [19]:
print(*file_lines, sep='\n')


X:1
T:Father O'Flynn
C:Trad.
M:6/8
L:1/8
K:Dmajor


In [20]:
# The result is good, so we can combine generating all the lines in one function
from muspy import Music
from music21.pitch import Pitch

def generate_header(music: Music) -> List[str]:
    header_lines = ["X:1"]
    header_lines.append(f"T:{music.metadata.title}")  # TODO: set filename as title if no other title
    creators = music.metadata.creators
    if (creators):
        header_lines.append(f"C:{','.join(creators)}")
    numerator = music.time_signatures[0].numerator
    denominator = music.time_signatures[0].denominator
    header_lines.append(f"M:{numerator}/{denominator}")  # TODO:  4/4 should be C and 2/2 should be C|
    header_lines.append(f"L:1/{denominator}")
    note = Pitch(music.key_signatures[0].root)
    mode = music.key_signatures[0].mode if music.key_signatures[0].mode is not None else ''
    header_lines.append(f"K:{note.name+mode}")
    return header_lines


Now comes a more difficult task: get the music notes and write them to the file's body. We can find notes in the Track object stored in the Music object. It contains a list of notes (Note) with corresponding pitch for each one encoded as MIDI number.

Once again we can use music21 Pitch object for coverting numbers to letter notes: 

In [21]:
notes = [Pitch(midi=note.pitch).name[0] for note in abc_music.tracks[0].notes]  # we don't want # symbols after notes
print(*notes, sep=' ')


A D A F D F A D E D C B A D C D E F G F D F E C A D A F D F A D E D C B A D C D E F G A D A F D F A D E D C B A D C D E F G F D F E C A D A F D F A D E D C B A D C D E F G F D D D


The letters we get are correct, but they are all capital letters. In our example we have small letters, too. 

Starting at middle {C} (4th octave), the notes in that octave are shown as

> CDEFGAB

The next note up is a {C} again – but to show it is in the higher octave (5th), that {C} is shown in lowercase as {c}. So going from middle {C} to the {B} one octave and seven notes above that is

> CDEFGABcdefgab

And we’re back at yet another {C} note. The next octave up is shown by an apostrophe immediately after the note name, like {c’}. The previous octave is shown by a comma immediately following the note name, eg {B,}. Now we can extend our scale further (and the range can be extended further by adding more commas or apostrophes):

> C,D,E,F,G,A,B,CDEFGABcdefgabc'd'e'f'g'a'b'

A simple algorithm converting notes from specific octaves into this notation looks as below:

In [22]:
def note_to_abc_str(note: Pitch):
    octave = note.octave
    if octave <= 4:
        comas = 4-octave
        return note.name[0] + ","*comas
    else:
        apostrophes = octave-5
        return note.name[0].lower() + "'"*apostrophes


In [23]:
notes = [note_to_abc_str(Pitch(midi=note.pitch)) for note in abc_music.tracks[0].notes]
print(*notes, sep=' ')


A d A F D F A d e d c B A d c d e f g f d f e c A d A F D F A d e d c B A d c d e f g A d A F D F A d e d c B A d c d e f g f d f e c A d A F D F A d e d c B A d c d e f g f d d d


We also have some barlines in the original example. They are stored in the Music object, so we can insert them between notes using the time attribute and a straightforward mergesort-like method:

In [24]:
def generate_note_body(music: Music) -> str:
    barlines = music.barlines
    bar_iter = 0
    notes = music.tracks[0].notes
    note_iter = 0
    note_str = ''
    while bar_iter < len(barlines) and note_iter < len(notes):
        if barlines[bar_iter].time <= notes[note_iter].time:
            note_str += ' | '
            bar_iter += 1
        else:
            note_str += note_to_abc_str(Pitch(midi=notes[note_iter].pitch))
            note_iter += 1
    while note_iter < len(notes):
        note_str += note_to_abc_str(Pitch(midi=notes[note_iter].pitch))
        note_iter += 1
    note_str += ' |'
    return note_str.lstrip()


In [25]:
note_str = generate_note_body(abc_music)
note_str


'| A | dAFDFA | dedcBA | dcdefg | fdfecA | dAFDFA | dedcBA | dcdefg | A | dAFDFA | dedcBA | dcdefg | fdfecA | dAFDFA | dedcBA | dcdefg | fddd |'

Much better! Let's check how our two functions work for another example (https://abcnotation.com/examples#notes-pitches):

> X:1 <br>
T:Notes / pitches <br>
M:C <br>
L:1/4 <br>
K:C treble <br>
C, D, E, F, | G, A, B, C | D E F G | A B c d | e f g a | b c' d' e' | f' g' a' b' |] <br>

In [26]:
abc_music1 = read_abc("abc_file_1.abc")
file_lines = generate_header(abc_music1)
file_lines.append(generate_note_body(abc_music1))
print(*file_lines, sep='\n')


X:1
T:Notes / pitches
M:4/4
L:1/4
K:C
| C,D,E,F, | G,A,B,C | DEFG | ABcd | efga | bc'd'e' | f'g'a'b' |


In [27]:
# Write these lines to a file
with open('result.abc', 'w') as f:
    for line in file_lines:
        f.write(f"{line}\n")


# We can also do this using new library function:
from muspy import write_abc

write_abc(path="muspy_result.abc", music=abc_music1)


Ways to expand this prototype:
- Finding repeated sections, marking them with |:
- Note length (n, /...)
- Frequency changing characters (#...)
- Notation specific details (like meter 4/4 = C)
- Key change mid-body
- Other abc syntax elements
- Splitting note body into multiple lines
- Adding barlines for readability
- Retrieving more metadata (if possible)

List of abc syntax elements: http://fileformats.archiveteam.org/wiki/ABC_(musical_notation)

Examples for testing and new features: https://abcnotation.com/examples 